**Flu Trends**

Applying the MultiOutput and Direct Strategies to the Flu Trends dataset and making true forecasts for multiple weeks beyond the training period. The forecast will have a 1-week lead time and an 8-week horizon. 

**Installing the libraries**


In [10]:
from pathlib import Path
from warnings import simplefilter

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor

simplefilter('ignore')

##Setting matplotlib defaults

plt.style.use('seaborn-v0_8-whitegrid')
plt.rc('figure', autolayout = True, figsize = (11,4))
plt.rc(
    'axes',
    labelweight = 'bold',
    labelsize = 'large',
    titleweight = 'bold',
    titlesize = 16,
    titlepad = 10,)

plot_params = dict(color = '0.75', style = ".-", markeredgecolor = '0.25', markerfacecolor = '0.25')

%config InlineBackend.figure_format = 'retina'

In [14]:
#Defining the helper function, plot_multistep

def plot_multistep(y, every = 1, ax = None, palette_kwargs = None):
    
    palette_kwargs_ = dict(palette = 'husl', n_colors = 16, desat = None)
    
    if palette_kwargs is not None:
        palette_kwargs_= palette_kwargs_.update(palette_kwargs)
    
    palette = sns.color_palette(**palette_kwargs_)

    if ax is None:
        fig, ax = plt.subplots()
    
    ax.set_prop_cycle(plt.cycler('color', palette)) #a color palette is set for the plot so each prediction series will have a different color

    for date, preds in y[::every].iterrows():
        preds.index = pd.period_range(start=date, periods = len(preds))
        preds.plot(ax=ax)
    return ax






